In [2]:
# import the packages we will need for this project
import pandas as pd
import numpy as np
import json
import requests
import api_keys

In [3]:
stations_df = pd.read_csv('stations_df.csv')
FS_KEY = api_keys.FOURSQUARE
YELP_KEY = api_keys.YELP

# Foursquare

## Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [18]:
def get_venues_fs(latitude, longitude, radius, categories):
    """
    Get venues from foursquare with a specified place type and coordinates.
    Args:
        latitude (float): latitude for query (must be combined with longitude)
        longitude (float): longitude for query (must be combined with latitude)
        api_key (str): foursquare API to use for query
        categories (str) : Foursquare-recognized place type. If not passed no place_type will be specified. Separate ids with commas
    
    Returns:
        response: response object from the requests library.
    """
    url = "https://api.foursquare.com/v3/places/search"
    
    params = {"categories": categories, # below, we will search for three different category types
              "radius":radius,
              "ll": f"{latitude},{longitude}",
              "limit":50, # this is the upper limit
              "fields": "rating,popularity"   # rating: A numerical rating (from 0.0 to 10.0) of the FSQ Place, based on user votes, likes/dislikes, tips sentiment, and visit data. Not all FSQ Places will have a rating.
                                              # popularity: A measure of the FSQ Place's popularity, by foot traffic. This score is on a 0 to 1 scale and uses a 6-month span of POI visits for a given geographic area.
             }
    # Create a dictionary for headers
    headers = {"Accept": "application/json"}
    # Add key with our API KEY
    headers['Authorization'] = FS_KEY 
    res = requests.get(url, params=params, headers=headers)
    foursquare_data = res.json()
    return foursquare_data

In [19]:
# test the function on bars (13003)
results = get_venues_fs(stations_df['latitude'][0], stations_df['longitude'][0], 1000, 13003)

## Combine the above flow into a for loop to obtain a count of different venues for the dataset and their category ID's
1. Bars - 13003 (cocktail bars, pubs, sports bars etc.)
2. Lodging - 19009 (Hotels, Motels, Hostels etc.)
3. Transportation Hubs - 19030 (Train stations, bus stations, rental car locations etc.)

In [20]:
# create a function to count the results for each category returned by the API call
def category_count(category):
    """
    Loop through the stations_df dataframe and return a list of counts of the number of items returned in the get_venues_fs function that corresponds with the input category
    
    Args:
        category (integer): the category integer of what POI we are searching for

    Returns:
        response: count_of_category: a list of the number of POI's which match the category integer from the get_venues_fs function
    """
    count_of_category = []

    for row in stations_df.itertuples(index=False): 
        
        latitude = row[8] # store the latitude of each station in a variable
        longitude = row[9] # store the longitude of each station in a variable
        
        venue_results = get_venues_fs(latitude, longitude, 1000, category) # call the get_venues_fs function for each station 
        normalized_results = pd.json_normalize(venue_results['results']) # store and normalize the results
        
        count_of_results = len(normalized_results) # count the number of rows in the normalized results
        count_of_category.append(count_of_results) # append the count to the empty list
    return count_of_category

In [21]:
# create a function to calculate the average popularity for the response from each category returned by the get_venues_fs function
def popularity_average(category):
    """
    Loop through the stations_df dataframe and return the calculated average popularity from the items returned in the get_venues_fs function that corresponds with the input category and append that value to a list
    
    Args:
        category (integer): the category integer of what POI we are searching for

    Returns:
        response: average_category_popularity: a list of the average popularity of POI's which match the category integer from the get_venues_fs function
    """
    average_category_popularity = []

    for row in stations_df.itertuples(index=False): 
        latitude = row[8]
        longitude = row[9]
        
        venue_results = get_venues_fs(latitude, longitude, 1000, category)
        normalized_results = pd.json_normalize(venue_results['results'])
        
        # Check if the 'popularity' column exists in the normalized results
        if 'popularity' in normalized_results.columns:
            average_popularity = normalized_results['popularity'].mean()
            average_category_popularity.append(average_popularity)
        else:
            # Case when the 'popularity' column is missing
            average_category_popularity.append(0)
        
    return average_category_popularity

In [22]:
# create a function to calculate the average rating for the response from each category returned by the get_venues_fs function
def rating_average(category):
    """
    Loop through the stations_df dataframe and return the calculated average ratings from the items returned in the get_venues_fs function that corresponds with the input category and append that value to a list
    
    Args:
        category (integer): the category integer of what POI we are searching for

    Returns:
        response: average_category_rating: a list of the average ranking of POI's which match the category integer from the get_venues_fs function
    """
    average_category_rating = []

    for row in stations_df.itertuples(index=False): 
        latitude = row[8]
        longitude = row[9]
        
        venue_results = get_venues_fs(latitude, longitude, 1000, category)
        normalized_results = pd.json_normalize(venue_results['results'])
        
        # Check if the 'rating' column exists in the normalized results
        if 'rating' in normalized_results.columns:
            average_rating = normalized_results['rating'].mean()
            average_category_rating.append(average_rating)
        else:
            # Case when the 'rating' column is missing
            average_category_rating.append(0)
        
    return average_category_rating

### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

#### I will be appending the count of each POI type, as well as the average popularity and average rating if the POI has either of these metrics

### Put your parsed results into a DataFrame

In [ ]:
# run the functions for bars
stations_df['num_nearby_bars'] = category_count(13003)
stations_df['avg_pop_bars'] = popularity_average(13003)
stations_df['avg_rat_bars'] = rating_average(13003)

In [ ]:
# run the functions for lodging
stations_df['num_nearby_lodging'] = category_count(19009)
stations_df['avg_pop_lodging'] = popularity_average(19009)
stations_df['avg_rat_lodging'] = rating_average(19009)

In [ ]:
# run the functions for transportation hubs
stations_df['num_nearby_trans_hubs'] = category_count(19030)
stations_df['avg_pop_trans_hubs'] = popularity_average(19030)
stations_df['avg_rating_trans_hubs'] = rating_average(19030)

In [ ]:
# save to dataframe
stations_df_fs = stations_df.to_csv('stations_df')

# Yelp

In [19]:
stations_df_fs = pd.read_csv('stations_df_fs.csv')

## Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [20]:
def get_venues_yelp(latitude, longitude, radius, categories):
    """
    Get venues from YELP with a specified place type and coordinates.
    Args:
        latitude (float): latitude for query (must be combined with longitude)
        longitude (float): longitude for query (must be combined with latitude)
        api_key (str): foursquare API to use for query
        categories (str) : Foursquare-recognized place type. If not passed no place_type will be specified. Separate ids with commas
    
    Returns:
        response: response object from the requests library.
    """
    url = "https://api.yelp.com/v3/businesses/search"
    
    params = {"latitude": latitude, # below, we will search for three different category types
              "longitude": longitude,
              "limit": 50,
              "categories": categories 
             }
    # Create a dictionary for headers
    headers = {"Accept": "application/json"}
    # Add key with our API KEY
    headers['Authorization'] = f"Bearer {YELP_KEY}" 
    res = requests.get(url, params=params, headers=headers)
    yelp_data = res.json()
    return yelp_data

### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

### Yelp Equivalents for Foursquare POI searches
1. Bars - 'bars' (Foursquare equivalent: 13003 (cocktail bars, pubs, sports bars etc.))
2. Hotels - 'hotels' (Foursquare equivalent: Lodging - 19009 (Hotels, Motels, Hostels etc.))
3. Transportation - 'transport' (Foursquare equivalent: Transportation Hubs - 19030 (Train stations, bus stations, rental car locations etc.))

In [6]:
# test with hotels
bars = get_venues_yelp(stations_df_fs['latitude'][0], stations_df_fs['longitude'][0], 1000, "hotels")
bars

{'businesses': [{'id': 'xJToH_Gy8RSKWqUPVUjwLw',
   'alias': 'the-hoxton-downtown-los-angeles-6',
   'name': 'The Hoxton - Downtown',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/G9QprxDFATluEZ5pRoDtqg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/the-hoxton-downtown-los-angeles-6?adjust_creative=d6_d6L4oKrfbGmQkOErdsg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=d6_d6L4oKrfbGmQkOErdsg',
   'review_count': 156,
   'categories': [{'alias': 'hotels', 'title': 'Hotels'},
    {'alias': 'venues', 'title': 'Venues & Event Spaces'}],
   'rating': 4.5,
   'coordinates': {'latitude': 34.039794135659584,
    'longitude': -118.25856086441767},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '1060 S Broadway',
    'address2': '',
    'address3': None,
    'city': 'Los Angeles',
    'zip_code': '90015',
    'country': 'US',
    'state': 'CA',
    'display_address': ['1060 S Broadway', 'Los Angeles, CA 90015']},
   'phone

In [7]:
results_yelp_bars = pd.json_normalize(bars['businesses'])
results_yelp_bars.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,xJToH_Gy8RSKWqUPVUjwLw,the-hoxton-downtown-los-angeles-6,The Hoxton - Downtown,https://s3-media1.fl.yelpcdn.com/bphoto/G9Qprx...,False,https://www.yelp.com/biz/the-hoxton-downtown-l...,156,"[{'alias': 'hotels', 'title': 'Hotels'}, {'ali...",4.5,[],...,34.039794,-118.258561,1060 S Broadway,,None,Los Angeles,90015,US,CA,"[1060 S Broadway, Los Angeles, CA 90015]"
1,u6wFTRE9RUY_LGNnIDbrzQ,omni-los-angeles-hotel-at-california-plaza-los...,Omni Los Angeles Hotel at California Plaza,https://s3-media1.fl.yelpcdn.com/bphoto/IWTBXO...,False,https://www.yelp.com/biz/omni-los-angeles-hote...,774,"[{'alias': 'hotels', 'title': 'Hotels'}, {'ali...",4.0,[],...,34.052599,-118.250504,251 S Olive St,,,Los Angeles,90012,US,CA,"[251 S Olive St, Los Angeles, CA 90012]"
2,1fGCTpM3zrcAgytrZA4mPQ,los-angeles-athletic-club-hotel-los-angeles,Los Angeles Athletic Club Hotel,https://s3-media1.fl.yelpcdn.com/bphoto/5MAjrl...,False,https://www.yelp.com/biz/los-angeles-athletic-...,101,"[{'alias': 'hotels', 'title': 'Hotels'}, {'ali...",4.0,[],...,34.046640,-118.254980,431 W 7th St,None,,Los Angeles,90014,US,CA,"[431 W 7th St, Los Angeles, CA 90014]"
3,Nh3L9PtJZTSGhfeAxKO3Gg,jw-marriott-los-angeles-l-a-live-los-angeles,JW Marriott Los Angeles L.A. LIVE,https://s3-media2.fl.yelpcdn.com/bphoto/1IqBhb...,False,https://www.yelp.com/biz/jw-marriott-los-angel...,941,"[{'alias': 'hotels', 'title': 'Hotels'}, {'ali...",4.0,[],...,34.045238,-118.266470,900 West Olympic Blvd,,,Los Angeles,90015,US,CA,"[900 West Olympic Blvd, Los Angeles, CA 90015]"
4,kvhmfXZ-JSkUUlsod_34OA,ace-hotel-downtown-los-angeles-los-angeles,Ace Hotel Downtown Los Angeles,https://s3-media3.fl.yelpcdn.com/bphoto/K4oavD...,False,https://www.yelp.com/biz/ace-hotel-downtown-lo...,738,"[{'alias': 'hotels', 'title': 'Hotels'}]",3.5,[],...,34.041741,-118.256794,929 S Broadway,,,Los Angeles,90015,US,CA,"[929 S Broadway, Los Angeles, CA 90015]"
5,1PSkXp8N6-cczfIBijTgNA,intercontinental-los-angeles-downtown-los-ange...,InterContinental Los Angeles Downtown,https://s3-media3.fl.yelpcdn.com/bphoto/lSkSCL...,False,https://www.yelp.com/biz/intercontinental-los-...,605,"[{'alias': 'hotels', 'title': 'Hotels'}]",3.5,[],...,34.050142,-118.260300,900 Wilshire Blvd,,None,Los Angeles,90017,US,CA,"[900 Wilshire Blvd, Los Angeles, CA 90017]"
6,gO4UMDCi7g0r_oiq_cUq1g,conrad-los-angeles-los-angeles,Conrad Los Angeles,https://s3-media2.fl.yelpcdn.com/bphoto/-mwmV-...,False,https://www.yelp.com/biz/conrad-los-angeles-lo...,86,"[{'alias': 'hotels', 'title': 'Hotels'}]",4.0,[],...,34.055360,-118.248790,100 South Grand Ave.,None,None,Los Angeles,90012,US,CA,"[100 South Grand Ave., Los Angeles, CA 90012]"
7,hMoFC3fmCEN-bRAB_c_Ujw,the-ritz-carlton-los-angeles-los-angeles-7,The Ritz Carlton Los Angeles,https://s3-media4.fl.yelpcdn.com/bphoto/Kim_iH...,False,https://www.yelp.com/biz/the-ritz-carlton-los-...,290,"[{'alias': 'hotels', 'title': 'Hotels'}, {'ali...",4.0,[],...,34.045263,-118.266620,900 W Olympic Blvd,,,Los Angeles,90015,US,CA,"[900 W Olympic Blvd, Los Angeles, CA 90015]"
8,Wh5ix84uh6MO5Rs1kI02gw,the-line-la-los-angeles,The LINE LA,https://s3-media1.fl.yelpcdn.com/bphoto/g8e-yH...,False,https://www.yelp.com/biz/the-line-la-los-angel...,1065,"[{'alias': 'hotels', 'title': 'Hotels'}, {'ali...",3.0,[],...,34.062130,-118.300995,3515 Wilshire Blvd,,,Los Angeles,90010,US,CA,"[3515 Wilshire Blvd, Los Angeles, CA 90010]"
9,oSqNlNpKImKF69GJ67-rCg,millennium-biltmore-los-angeles-los-angeles-4,Millennium Biltmore - Los Angeles,https://s3-media3.fl.yelpcdn.com/bphoto/rfVGM7...,False,https://www.yelp.com/biz/millennium-biltmore-l...,1308,"[{'alias': 'hotels', 'title': 'Hotels'}, {'ali...",3.0,[],...,34.049730,-118.253890,506 S Grand Ave,,,Los Angeles,90071,

### Repeat the functions created to loop through the Foursquare results for YELP

In [5]:
#def category_count_yelp(category):
    """
    Loop through the stations_df_fs dataframe and return a list of counts of the number of items returned in the get_venues_fs function that corresponds with the input category
    
    Args:
        category (integer): the category integer of what POI we are searching for

    Returns:
        response: count_of_category: a list of the number of POI's which match the category integer from the get_venues_fs function
    """
    count_of_category = []

    for row in stations_df_fs.itertuples(index=False): 
        
        latitude = row[8] # store the latitude of each station in a variable
        longitude = row[9] # store the longitude of each station in a variable
        
        venue_results = get_venues_yelp(latitude, longitude, 1000, category) # call the get_venues_fs function for each station 
        normalized_results = pd.json_normalize(venue_results['businesses']) # store and normalize the results
        
        count_of_results = len(normalized_results) # count the number of rows in the normalized results
        count_of_category.append(count_of_results) # append the count to the empty list
    
    return count_of_category

### After calling this function on all three categories, the results showed that the return limit of 50 results was reached for each station and therefore, the analysis will focus the API calls on the other YELP functions.

In [21]:
def average_rating_yelp(category):
    """
    Loop through the stations_df_fs dataframe and return the calculated average ratings from the items returned in the get_venues_yelp function that corresponds with the input category and append that value to a list
    
    Args:
        category (string): the category string of businesses we are searching for. For multiple categories, input a comma-separated string list "hotel,bars"

    Returns:
        response: average_category_rating: a list of the average ranking of businesses which match the category integer from the get_venues_yelp function
    """
    average_category_rating = []

    for row in stations_df_fs.itertuples(index=False): 
        
        latitude = row[8] # store the latitude of each station in a variable
        longitude = row[9] # store the longitude of each station in a variable
        
        venue_results = get_venues_yelp(latitude, longitude, 1000, category) # call the get_venues_fs function for each station 
        normalized_results = pd.json_normalize(venue_results['businesses']) # store and normalize the results
        
        # Check if the 'rating' column exists in the normalized results
        if 'rating' in normalized_results.columns:
            average_rating = normalized_results['rating'].mean()
            average_category_rating.append(average_rating)
        else:
            # Case when the 'rating' column is missing
            average_category_rating.append(0)
        
    return average_category_rating

In [20]:
#def average_reviews_yelp(category):
    """
    Loop through the stations_df_fs dataframe and return the calculated average number of reviews from the items returned in the get_venues_yelp function that corresponds with the input category and append that value to a list
    
    Args:
        category (string): the category string of businesses we are searching for ('transport'). For multiple categories, input a comma-separated string list ("hotel,bars")

    Returns:
        response: average_count_reviews: a list of the average number of reviews for the businesses which match the category integer from the get_venues_yelp function
    """
    average_count_reviews = []

    for row in stations_df_fs.itertuples(index=False): 
        
        latitude = row[8] # store the latitude of each station in a variable
        longitude = row[9] # store the longitude of each station in a variable
        
        venue_results = get_venues_yelp(latitude, longitude, 1000, category) # call the get_venues_fs function for each station 
        normalized_results = pd.json_normalize(venue_results['businesses']) # store and normalize the results
        
        # Check if the 'review_count' column exists in the normalized results
        if 'review_count' in normalized_results.columns:
            average_reviews = normalized_results['review_count'].mean()
            average_count_reviews.append(average_reviews)
        else:
            # Case when the 'review_count' column is missing
            average_count_reviews.append(0)
        
    return average_count_reviews

In [ ]:
# average_num_yelp_reviews_bar = average_reviews_yelp('bars')

### Put your parsed results into a DataFrame

In [ ]:
# call the function on the bar category
average_yelp_rating_bars = average_rating_yelp('bars')

In [ ]:
# append the results of the function above to the dataframe as a column for analysis
stations_df_fs['average_yelp_rating_bar'] = average_rating_yelp_bars

#### API limit reached for day, save dataframe and resume when API limit resets

In [29]:
stations_df_fs_yelp = stations_df_fs.to_csv('stations_df_fs_yelp.csv')

#### API limit reset, read in saved dataframe and call YELP functions for remaining categories

In [10]:
#read in the csv
stations_df_fs_yelp = pd.read_csv('stations_df_fs.csv')

In [9]:
# test the function for hotels category
average_yelp_rating_hotels = average_rating_yelp('hotels')

In [13]:
# call the function on transport category
average_yelp_rating_transport = average_rating_yelp('transport')

In [11]:
# append the results of the hotel function to the dataframe
stations_df_fs_yelp['average_yelp_rating_hotels'] = average_yelp_rating_hotels

In [14]:
# append the results of the hotel function to the dataframe
stations_df_fs_yelp['average_yelp_rating_transport'] = average_yelp_rating_transport

In [24]:
# save the dataframe to a .csv
stations_df_fs_yelp_1 = stations_df_fs_yelp.to_csv('stations_df_fs_yelp_1.csv')

# Comparing Results

## Which API provided you with more complete data? Provide an explanation. 

### Foursquare

#### Pros:
In the context of this analysis, I believe that the popularity metric, which only Foursquare provides, is very powerful. When factoring in bike rentals, this mode of transportation is closer to walking than it is to vehicular travel, and therefore this shows a correlation. I also believe that because the metric is only for the last six months, this allows the metric to be more fluid and accurate. For example, imagine a bar opened 5 years ago and as of today, the bar has a 9/10 rating on Foursquare. However, under closer scrutiny, most good reviews came more than three years ago, and the average rating in the last two years is lower than before. This allows us to compare the normalized rating and popularity values later to see how well they compare.
#### Cons:
Not having a 'reviews' column to show the number of reviews that went into calculating the 'rating' column makes the 'rating' a black box; there could be only one review for a POI. 

### YELP

#### Pros:
The reviews column allows for filtering out results that don't meet a specified threshold if required. Additionally, there are more businesses on YELP vs. Foursquare, which better represents the real world.

#### Cons:
The limitations of the YELP API are much more restrictive and prevent tuning of the request with the 500 API calls per day limit. The ability to call the Foursquare API with fewer restrictions is very helpful to iteratively build functions by testing each step. Additionally, the rating column for YELP is on a scale from 0 - 5 compared to the Foursquare scale of 0 - 10 which allows for better precision.

Considering the above, I believe the Foursquare API provided me with more complete data.

## Get the top 10 restaurants according to their rating

In [55]:
def get_restaurants_fs(latitude, longitude, radius, categories):
    """
    Get restaurants from foursquare with a specified place type and coordinates.
    Args:
        latitude (float): latitude for query (must be combined with longitude)
        longitude (float): longitude for query (must be combined with latitude)
        api_key (str): foursquare API to use for query
        categories (str) : Foursquare-recognized place type. If not passed no place_type will be specified. Separate ids with commas
    
    Returns:
        response: response object from the requests library.
    """
    url = "https://api.foursquare.com/v3/places/search"
    
    params = {"categories": categories, # below, we will search for three different category types
              "radius":radius,
              "ll": f"{latitude},{longitude}",
              "limit":50, # this is the upper limit
              "fields": "fsq_id,name,rating"  
            }
    # Create a dictionary for headers
    headers = {"Accept": "application/json"}
    # Add key with our API KEY
    headers['Authorization'] = FOURSQUARE_KEY 
    res = requests.get(url, params=params, headers=headers)
    foursquare_data = res.json()
    return foursquare_data

In [57]:
# using the above function, loop through each station, searching for restaurants, append the results to an empty list, then concatenate to the new dataframe
restaurants = pd.DataFrame() # empty dataframe that will have values concatenated to it
list_of_restaurants = [] # empty list for appending results

for row in stations_df.itertuples(index=False):
    latitude = row[8]
    longitude = row[9]
            
    venue_results = get_restaurants_fs(latitude, longitude, 1000, FOURSQUARE_KEY, 13065)
    normalized_results = pd.json_normalize(venue_results['results'])
    list_of_restaurants.append(normalized_results)

restaurants = pd.concat(list_of_restaurants)    

In [96]:
# Fill the NaN values for restaurants with zeros
restaurants = restaurants.fillna(0)
restaurants

,fsq_id,name,rating
0,60c184c9528f822158a147e8,Afuri Ramen,8.9
1,55fccaa5498ec525f32b644d,Everson Rocye Bar,9.1
2,4a15b070f964a520b9781fe3,Tony's Saloon,8.6
3,4e0e69c445ddc2c6d1780ab2,Pizzanista,8.4
4,5b549f2ec97f28002cb31a5f,Guerrilla Tacos,8.5
...,...,...,...
37,63d4b6beea0fa64ea1f18ef1,Mun Korean Steakhouse,0.0
38,4c9e3729d3c2b60cb8f7c8bc,La Poblanita,0.0
39,62bf97634432bf416e57b6ba,Kunnai Thai Restaurant,0.0
40,7c75265c14f0448ae6e60e30,Mariscos Michoacan,0.0


In [93]:
# check for duplicate values
restaurants.duplicated().sum()

6950

In [94]:
# the majority of the restaurants are duplicates, drop from the analysis
clean_restaurants = restaurants.drop_duplicates(subset=['fsq_id'], keep='first')
# remove the fsq_id column
clean_restaurants = clean_restaurants.drop(['fsq_id'],axis=1)
clean_restaurants

,name,rating
0,Afuri Ramen,8.9
1,Everson Rocye Bar,9.1
2,Tony's Saloon,8.6
3,Pizzanista,8.4
4,Guerrilla Tacos,8.5
...,...,...
33,Citizen Sprout,0.0
34,Citrin,0.0
35,Tiato Kitchen & Venue,0.0
36,Cafe Lucy,0.0


In [95]:
# sort the values by 'rating' and return the top 10
top_10_restaurants = clean_restaurants.sort_values('rating', ascending=False).head(10)

#### The top 10 restaurants from Foursquare results within 1000m of all city bik.es stations are:

In [88]:
top_10_restaurants

,name,rating
7,Salt & Straw,9.4
8,Saffron & Rose Ice Cream,9.3
11,Guisados,9.3
2,Firestone Walker - The Propagator,9.3
11,Superba Food + Bread,9.3
0,Porto's Bakery & Cafe,9.3
9,Bavel,9.3
35,Cobi's,9.3
24,Walt Disney Concert Hall,9.3
7,Hatchet Hall,9.2
